In [10]:
from fmri_preprocessing import vectorized_correlation
from utils import find_repo_root
from utils import load_dict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import os
from sklearn.preprocessing import StandardScaler
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt

### Function Definitions

In [2]:


# fast linear regression function with Pytorch (taken from CCN2021_Algonauts.ipynb)
# class OLS_pytorch(object):
#     def __init__(self,use_gpu=False):
#         self.coefficients = []
#         self.use_gpu = use_gpu
#         self.X = None
#         self.y = None
# 
#     def fit(self,X,y):
#         if len(X.shape) == 1:
#             X = self._reshape_x(X)
#         if len(y.shape) == 1:
#             y = self._reshape_x(y)
# 
#         X =  self._concatenate_ones(X)
# 
#         X = torch.from_numpy(X).float()
#         y = torch.from_numpy(y).float()
#         if self.use_gpu:
#             X = X.cuda()
#             y = y.cuda()
#         XtX = torch.matmul(X.t(),X)
#         Xty = torch.matmul(X.t(),y.unsqueeze(2))
#         XtX = XtX.unsqueeze(0)
#         XtX = torch.repeat_interleave(XtX, y.shape[0], dim=0)
#         betas_cholesky, _ = torch.solve(Xty, XtX)
# 
#         self.coefficients = betas_cholesky
# 
#     def predict(self, entry):
#         if len(entry.shape) == 1:
#             entry = self._reshape_x(entry)
#         entry =  self._concatenate_ones(entry)
#         entry = torch.from_numpy(entry).float()
#         if self.use_gpu:
#             entry = entry.cuda()
#         prediction = torch.matmul(entry,self.coefficients)
#         prediction = prediction.cpu().numpy()
#         prediction = np.squeeze(prediction).T
#         return prediction
# 
#     def _reshape_x(self,X):
#         return X.reshape(-1,1)
# 
#     def _concatenate_ones(self,X):
#         ones = np.ones(shape=X.shape[0]).reshape(-1,1)
#         return np.concatenate((ones,X),1)

# def predict_fmri_fast(train_activations, test_activations, train_fmri,use_gpu=False):
#     """This function fits a linear regressor using train_activations and train_fmri,
#     then returns the predicted fmri_pred_test using the fitted weights and
#     test_activations.
#     Parameters
#     ----------
#     train_activations : np.array
#         matrix of dimensions #train_vids x #pca_components
#         containing activations of train videos.
#     test_activations : np.array
#         matrix of dimensions #test_vids x #pca_components
#         containing activations of test videos
#     train_fmri : np.array
#         matrix of dimensions #train_vids x  #voxels
#         containing fMRI responses to train videos
#     use_gpu : bool
#         whether to use gpu or not.
#     Returns
#     -------
#     fmri_pred_test: np.array
#         matrix of dimensions #test_vids x  #voxels
#         containing predicted fMRI responses to test videos .
#     """
# 
#     reg = OLS_pytorch(use_gpu)
#     reg.fit(train_activations,train_fmri.T)
#     fmri_pred_test = reg.predict(test_activations)
#     return fmri_pred_test

In [3]:
def get_fmri(fmri_dir, ROI):
    """This function loads fMRI data into a numpy array for to a given ROI.
    Parameters
    ----------
    fmri_dir : str
        path to fMRI data. Relates to a specific subject.
    ROI : str
        name of ROI.
    Returns
    -------
    np.array
        matrix of dimensions #train_vids x #repetitions x #voxels
        containing fMRI responses to train videos of a given ROI
    """

    # Loading ROI data
    # @Julian: adjust path as soon as preprocessing is done
    ROI_file = os.path.join(fmri_dir, ROI + ".pkl")
    ROI_data = load_dict(ROI_file)

    # averaging ROI data across repetitions
    ROI_data_train = np.mean(ROI_data["train"], axis = 1)
    if ROI == "WB":
        voxel_mask = ROI_data['voxel_mask']
        return ROI_data_train, voxel_mask

    return ROI_data_train

In [4]:
# Dense Layer
def train_model(X_train, X_val, y_train, y_val, num_epochs = 1000):
    """
    conducts the training for a particular layer, subject & ROI
    :param X_train: training data (feature map PCs from first 800 videos from a particular layer)
    :param X_val: validation data (feature map PCs from videos 801-900 from a particular layer)
    :param y_train: training labels (scans for first 800 videos for the particular subject & ROI)
    :param y_val: training labels (scans for videos 801-900 for the particular subject & ROI)
    :param num_epochs: number of epochs for training
    :return: model parameters & validation accuracy
    """
    
    # specify number of neurons per layer, depending on the number of inputs from a particular layer and number of output voxels
    input_neurons = X_train.shape[1]
    hidden1_neurons = y_train.shape[1] + (X_train.shape[1] - y_train.shape[1])*(2/3)
    hidden2_neurons = y_train.shape[1] + (X_train.shape[1] - y_train.shape[1])*(1/3)
    output_neurons = y_train.shape[1]
    
    # model construction: 2 hidden layers
    model = Sequential([
        Dense(hidden1_neurons, input_shape=(input_neurons,), activation='relu'),
        Dense(hidden2_neurons, activation='relu'),
        Dense(output_neurons)
    ])
    
    # Compiling the model
    adam = Adam(lr=0.001)
    model.compile(optimizer=adam, loss='mean_squared_error')
    
    # ToDo: define the custom evaluation metric (via keras.backend)
    
    # Training the model
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_data=(X_val, y_val))
    
    # extract validation accuracy
    validation_accuracy = history.history['val_accuracy']

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    return model, validation_accuracy

In [5]:
# train-val-test split, fitting linear regression model, visualize voxel predictions
def perform_encoding(pca_dir, fmri_dir,results_dir, sub, layer, ROI = 'WB', mode = 'val', visualize_results = True, batch_size=1000):
    if torch.cuda.is_available():
      use_gpu = True
    else:
      use_gpu = False
    
    # Load activations (PCA outputs)
    # ToDo: build get_activations
    # @Julian: change directory names as soon as PCA is done
    pca_dir = os.path.join("/content/activations" )
    train_activations,test_activations = get_activations(pca_dir, layer)
    
    # Load fMRI data (labels)
    # ToDO: adjust directory as soon as fMRI preprocessing is done
    if ROI == "WB":
      track = "full_track"
    else:
      track = "mini_track"
    fmri_dir = os.path.join(fmri_dir, track)
    sub_fmri_dir = os.path.join(fmri_dir, sub)
    if track == "full_track":
      fmri_train_all,voxel_mask = get_fmri(sub_fmri_dir,ROI)
    else:
      fmri_train_all = get_fmri(sub_fmri_dir,ROI)
    num_voxels = fmri_train_all.shape[1]
    
    
    # Creating data splits
    if mode == 'val':
      # split labels
      train_activations = train_activations[800:900,:]
      val_activations = train_activations[:800,:]
      
      
      fmri_train = fmri_train_all[:800,:]
      fmri_val = fmri_train_all[800:900,:]
      pred_fmri = np.zeros_like(fmri_val)
      pred_fmri_save_path = os.path.join(results_dir, ROI + '_val.npy')
      
      trained_model, val_acc = train_model(train_activations, val_activations, fmri_train, fmri_val)
    
    # ToDo @Marcel: implement predictions on test set
    # else:
    #     fmri_train = fmri_train_all
    #     num_test_videos = 102
    #     pred_fmri = np.zeros((num_test_videos,num_voxels))
    #     pred_fmri_save_path = os.path.join(results_dir, ROI + '_test.npy')
    ######################################

    # iter = 0
    # 
    # while iter < num_voxels-batch_size:
    #     pred_fmri[:,iter:iter+batch_size] = predict_fmri_fast(train_activations,test_activations,fmri_train[:,iter:iter+batch_size], use_gpu = use_gpu)
    #     iter = iter+batch_size
    # pred_fmri[:,iter:] = predict_fmri_fast(train_activations,test_activations,fmri_train[:,iter:iter+batch_size], use_gpu = use_gpu)
    # if mode == 'val':
    #   score = vectorized_correlation(fmri_val,pred_fmri)
    #   ################################################
    # 
    #   nii_save_path =  os.path.join(results_dir, ROI + '_val.nii')
    #   ######## Result visualization ################
    #   if track == "full_track" and visualize_results:
    #       visual_mask_3D = np.zeros((78,93,71))
    #       visual_mask_3D[voxel_mask==1]= score
    #       brain_mask = './example.nii'
    #       saveasnii(brain_mask,nii_save_path,visual_mask_3D)
    #       plotting.plot_glass_brain(nii_save_path,plot_abs=False,
    #                         title='Correlation for ' + sub+ ' and ' + layer,
    #                         display_mode='lyr',colorbar=True,vmin=-1,vmax=1)
    
    ################################################
    # return score.mean()
    
    # np.save(pred_fmri_save_path, pred_fmri)



IndentationError: unexpected indent (853691021.py, line 53)

Shape of fmri data for sub01: (1000, 3, 18222)
Shape of fmri data for sub02: (1000, 3, 21573)
Shape of fmri data for sub03: (1000, 3, 15225)
Shape of fmri data for sub04: (1000, 3, 19445)
Shape of fmri data for sub05: (1000, 3, 13340)
Shape of fmri data for sub06: (1000, 3, 19818)
Shape of fmri data for sub07: (1000, 3, 10836)
Shape of fmri data for sub08: (1000, 3, 12347)
Shape of fmri data for sub09: (1000, 3, 17570)
Shape of fmri data for sub10: (1000, 3, 12950)


'C:\\Users\\julia\\OneDrive - CBS - Copenhagen Business School\\Documents\\Master\\Semester3\\AdvancedML\\Brainvision_Project'

In [ ]:
def get_activations(activations_dir, layer_name):
    """This function loads neural network features/activations (preprocessed using PCA) into a
    numpy array according to a given layer.
    Parameters
    ----------
    activations_dir : str
        Path to PCA processed Neural Network features
    layer_name : str
        which layer of the neural network to load,
    Returns
    -------
    train_activations : np.array
        matrix of dimensions #train_vids x #pca_components
        containing activations of train videos
    test_activations : np.array
        matrix of dimensions #test_vids x #pca_components
        containing activations of test videos
    """
    # @Julian: adjust paths as soon as created
    
    # numpy arrays of the PCA results
    # "train" contains PCA outputs of all videos, "test" is for the validation set
    train_file = os.path.join(activations_dir,"train_" + layer_name + ".npy")
    test_file = os.path.join(activations_dir,"test_" + layer_name + ".npy")
    train_activations = np.load(train_file)
    test_activations = np.load(test_file)
    
    # standardize PCA output
    scaler = StandardScaler()
    train_activations = scaler.fit_transform(train_activations)
    test_activations = scaler.fit_transform(test_activations)

    return train_activations, test_activations

In [1]:
# # get prediction results
# 
# # list of all subjects
# subs = ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]
# 
# #root fmri directory
# fmri_dir = './participants_data_v2021'
# 
# # path where to save predictions
# prediction_dir = './prediction'
# 
# # list of ROIs
# ROIs = ["V1", "V2","V3", "V4", "LOC", "EBA", "FFA","STS", "PPA"]
# 
# # Initializing dictionary to store results
# results_to_plot = {}
# 
# # Which layer of model for prediction
# layer = 'layer4'
# 
# # Which track to predict
# track = 'mini_track' 
# 
# model_name = 'resnet50'
# 
# results_to_plot[model_name] = {}
# # path to activations directory
# activations_dir = "./activations_" + model_name # adapt to out folder structure
# 
# # for loop running over all ROIs
# for ROI in ROIs:
#     results_to_plot[model_name][ROI] = 0
#     # loop over subjects
#     for sub in subs:
#         # creating results directory to save results
#         results_dir = os.path.join(prediction_dir,model_name, layer,\
#                             track, sub)
#         if not os.path.exists(results_dir):
#             os.makedirs(results_dir)
# 
#         # encoding
#         results_to_plot[model_name][ROI] += perform_encoding(activations_dir, fmri_dir,\
#                                   results_dir, sub, layer, ROI=ROI)
#     # averaging the correlation across subjects
#     results_to_plot[model_name][ROI]/=len(subs)

In [ ]:
# plot results
pd.DataFrame(results_to_plot).plot(kind='bar')

plt.title("Correlation with validation set: " + layer)
plt.ylabel("Correlation")
plt.xlabel("ROIs")
plt.ylim([0,0.5])
plt.show()

### Calculations

In [ ]:
# define repo root (necessary for runs in Ucloud)
repo_root = find_repo_root()
pca_dir = os.path.join(repo_root, "PCA")
print(pca_dir)

# specify layer manually: ["stage_1", "stage_2", "stage_3", "stage_4", "stage_5", "final"]
layer = 'layer4'

subs = ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]
ROIs = ["WB", "V1", "V2","V3", "V4", "LOC", "EBA", "FFA","STS", "PPA"]
for sub in subs:
  # leaving ROI choice in the code for now - can be removed later
  for ROI in ROIs:
    if ROI == "WB":
        track = "full_track"
    else:
        track = "mini_track"

    # ToDo: implement some saving of prediction scores
    # specify directory to save prediction scores
    # results_dir = os.path.join(prediction_dir,model, layer, track, sub)  
    # if not os.path.exists(results_dir):
    #   os.makedirs(results_dir)
    print ("Starting ROI: ", ROI, "sub: ",sub)
    perform_encoding(pca_dir, fmri_dir,\
                     results_dir, sub, layer,\
                     ROI=ROI,mode='val')  # mode: 'val' for model training/selection, 'test' for obtaining evaluation metrics
    print ("Completed ROI: ", ROI, "sub: ",sub)
    print("----------------------------------------------------------------------------")

In [ ]:
# def calculate_vectorized_correlation(x, y):
#     """
#     calculate evaluation score (per voxel)
#     :param: x - prediction voxel activations; y - actual voxel activations
#     :return: evaluation score, maybe other evaluation metrics can be added here as well
#     """
#     dim = 0
# 
#     centered_x = x - x.mean(axis=dim, keepdims=True)
#     centered_y = y - y.mean(axis=dim, keepdims=True)
# 
#     covariance = (centered_x * centered_y).sum(axis=dim, keepdims=True)
# 
#     covariance = covariance / (x.shape[dim])
# 
#     x_std = x.std(axis=dim, keepdims=True) + 1e-8
#     y_std = y.std(axis=dim, keepdims=True) + 1e-8
# 
#     corr = covariance / (x_std * y_std)
# 
#     return corr.ravel()